In [2]:
import glob
import os
from pptx import Presentation
import xlsxwriter
from pptx.enum.action import PP_ACTION
from pptx.enum.shapes import MSO_SHAPE_TYPE
import urllib.request
from datetime import date
import time
import requests
import multiprocessing
from multiprocessing.pool import ThreadPool
# from clint.textui import progress

def url_response(addr):
    r = requests.get(addr, stream = True)
    with open(file_name, 'wb') as f:
        for ch in r:
            f.write(ch)

# 이미지 저장
def write_image(shape,creative_id):
    global n
    global image_filename
    image = shape.image
    image_bytes = image.blob
    image_filename = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + creative_id  + '.jpg'
    n += 1
    print(image_filename)
    with open(image_filename, 'wb') as f:
        f.write(image_bytes)  

n=1
image_filename=""
creative_id=""
e_row = 1   # 2번째 행부터 시작
e_col = 0
stime = time.time()
url_list = []

os.mkdir("OUTPUT")
f = open('OUTPUT/' + 'check.txt','w')
today = str(date.today().strftime('%Y%m%d'))
workbook = xlsxwriter.Workbook('OUTPUT/' + '광고_RAW DATA_'+ today +'.xlsx')
worksheet = workbook.add_worksheet('data')

#엑셀 파일 1행 셋팅
cell_format = workbook.add_format()
cell_format.set_bg_color('#b7dee8')
cell_format.set_bottom()
worksheet.write(0,0,'Creative ID',cell_format)
worksheet.write(0,1,'JPG 파일이 저장된 경로',cell_format)
worksheet.write(0,2,'JPG 파일에 연결된 Link',cell_format)
worksheet.write(0,3,'Title',cell_format)
worksheet.write(0,4,'Media',cell_format)
worksheet.write(0,5,'Description',cell_format)
worksheet.write(0,6,'Country',cell_format)
worksheet.write(0,7,'Product',cell_format)
worksheet.write(0,8,'Category (Full Hierarchy):',cell_format)
worksheet.write(0,9,'Date From',cell_format)
worksheet.write(0,10,'Date To',cell_format)
worksheet.write(0,11,'Channel',cell_format)
#     worksheet.write(0,12,'First URL',cell_format)
#     worksheet.write(0,13,'Publication',cell_format)
worksheet.write(0,12,'Endline',cell_format)

print('pptx를 옮겨쓰는 중이에요~')
    
for file in glob.glob("./*.pptx"):
    fileName_only = os.path.basename(file) #파일명
    filedir = os.path.abspath(file) #파일 경로
    n = 1    
#     worksheet = workbook.add_worksheet(name = fileName_only) #파일명으로 시트 추가
    
    prs = Presentation(file)

    for slide in prs.slides:
        slide_num = prs.slides.index(slide)
        e_col = 0
        url_list = []
        
        if(slide_num==0): #첫번째 슬라이드 제외
            continue
        for shape in slide.shapes:
#             print(shape.shape_type)
#             print(slide_num)
            #테이블 내용 저장
            if shape.has_table:
                tbl = shape.table
                row_count = len(tbl.rows)
            
                for r in range(1, row_count):            #첫줄에 공백이 있음 & 첫 슬라의 prepared for 제거
                    if tbl.cell(r,0).text in "Title:":
                        worksheet.write(e_row,3,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Media:":  
                        worksheet.write(e_row,4,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Description:":  
                        worksheet.write(e_row,5,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Country:":  
                        worksheet.write(e_row,6,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Product:":  
                        worksheet.write(e_row,7,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Category (Full Hierarchy):":  
                        worksheet.write(e_row,8,tbl.cell(r,1).text)
    #                 elif tbl.cell(r,0).text == "Duration:":        
    #                     worksheet.write(e_row,e_col,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Date From/To:": 
                        if "-" in tbl.cell(r,1).text:
                            date_start = tbl.cell(r,1).text.split(' - ')[0]
                            date_end = tbl.cell(r,1).text.split(' - ')[1]
                            worksheet.write(e_row,9,date_start)
                            worksheet.write(e_row,10,date_end)
                        else:
                            worksheet.write(e_row,9,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Channel:":  
                        worksheet.write(e_row,11,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "First URL:":  
                        worksheet.write(e_row,11,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Publication:":  
                        worksheet.write(e_row,11,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Endline:":  
                        worksheet.write(e_row,12,tbl.cell(r,1).text)
                    elif tbl.cell(r,0).text in "Creative ID:":  
                        creative_id = tbl.cell(r,1).text
                        worksheet.write(e_row,0,creative_id)
                    elif tbl.cell(r,0).text in "Duration:":  #Duration은 안쓰나봅니다. 
                        None
                    else:
                        f.write(fileName_only+'\t')
                        f.write(tbl.cell(r,0).text+'\n')
                        
                if (ctype in 'image/jpeg') or (ctype in 'text/html'):
                    os.rename('OUTPUT/' + fileName_only + '_' + str(n-1) + '_' + '.jpg','OUTPUT/' + fileName_only + '_' + str(n-1) + '_' + creative_id  + '.jpg')
                elif ctype in 'video/mp4':
                    os.rename('OUTPUT/' + fileName_only + '_' + str(n-1) + '_'  + '.mp4','OUTPUT/' + fileName_only + '_' + str(n-1) + '_' + creative_id  + '.mp4')

#             #하이퍼링크 저장            
#             if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
#                 click_action = shape.click_action                
#                 if (click_action.action == PP_ACTION.HYPERLINK):
#                     addr = click_action.hyperlink.address
#                     worksheet.write(e_row,2,addr)
#                     url_list.append(addr)
#                     ctype = urllib.request.urlopen(addr).info().get_content_type()
# #                     print(ctype)
#                     if (ctype in 'image/jpeg') or (ctype in 'text/html'):
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.jpg'
#                     elif ctype in 'video/mp4':
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.mp4'
#                     n += 1                    
                    
            #하이퍼링크 저장            
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                click_action = shape.click_action                
                if (click_action.action == PP_ACTION.HYPERLINK):
                    addr = click_action.hyperlink.address
                    worksheet.write(e_row,2,addr)
                    ctype = urllib.request.urlopen(addr).info().get_content_type()
#                     print(ctype)
                    if (ctype in 'image/jpeg') or (ctype in 'text/html'):
                        file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.jpg'
                    elif ctype in 'video/mp4':
                        file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.mp4'
                    urllib.request.urlretrieve(addr,file_name)
                    n += 1

#             #하이퍼링크 저장  size10 -35s size 1024 -59s 1024*1024 33s        
#             if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
#                 click_action = shape.click_action                
#                 if (click_action.action == PP_ACTION.HYPERLINK):
#                     addr = click_action.hyperlink.address
#                     worksheet.write(e_row,2,addr)
#                     ctype = urllib.request.urlopen(addr).info().get_content_type()
# #                     print(ctype)
#                     if (ctype in 'image/jpeg') or (ctype in 'text/html'):
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.jpg'
#                     elif ctype in 'video/mp4':
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.mp4'
#                     r = requests.get(addr, stream = True)
#                     with open(file_name, 'wb') as f:
#                         for chunk in r.iter_content(chunk_size=1024*1024*1024):
#                             if chunk:
#                                 f.write(chunk)
#                     n += 1   
                    
#             #하이퍼링크 저장 23.711           
#             if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
#                 click_action = shape.click_action                
#                 if (click_action.action == PP_ACTION.HYPERLINK):
#                     addr = click_action.hyperlink.address
#                     worksheet.write(e_row,2,addr)
#                     ctype = urllib.request.urlopen(addr).info().get_content_type()
# #                     print(ctype)
#                     if (ctype in 'image/jpeg') or (ctype in 'text/html'):
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.jpg'
#                     elif ctype in 'video/mp4':
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.mp4'
#                     r = requests.get(addr, stream = True)
#                     with open(file_name, 'wb') as f:
#                         for chunk in r:
#                             f.write(chunk)
#                     n += 1 
                    
#             #하이퍼링크 저장 22.139           
#             if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
#                 click_action = shape.click_action                
#                 if (click_action.action == PP_ACTION.HYPERLINK):
#                     addr = click_action.hyperlink.address
#                     worksheet.write(e_row,2,addr)
#                     ctype = urllib.request.urlopen(addr).info().get_content_type()
# #                     print(ctype)
#                     if (ctype in 'image/jpeg') or (ctype in 'text/html'):
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.jpg'
#                     elif ctype in 'video/mp4':
#                         file_name = 'OUTPUT/' + fileName_only + '_' + str(n) + '_' + '.mp4'
#                     r = requests.get(addr)
#                     open(file_name, 'wb').write(r.content)
#                     n += 1                      
                    
#             #이미지 저장
#             if (creative_id != "") & (shape.shape_type == MSO_SHAPE_TYPE.PICTURE):
#                 write_image(shape,creative_id)
            
        #파일 경로 저장
        worksheet.write(e_row,1,filedir)
        
#         pool = multiprocessing.Pool(processes=2)
#         pool.map(url_response, url_list)
#         pool.close()
#         pool.join()
                    
        if(slide_num>0):                             #첫 슬라이드 제외
            e_row += 1

print("진행 시간:" + str(time.time() - stime))
print("완료! enter를 눌러 창을 닫아주세요 :) ")
a = input()            
workbook.close()
f.close()

pptx를 옮겨쓰는 중이에요~
진행 시간:21.69624090194702
진행 시간:30.55874800682068
진행 시간:33.389909744262695
진행 시간:43.19347047805786
진행 시간:62.49957466125488
진행 시간:74.80127835273743
진행 시간:74.80927872657776
완료! enter를 눌러 창을 닫아주세요 :) 

